<h1>IS4242 Group Project</h1>

<b>Import necessary libraries</b>

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score, precision_score, recall_score, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import seaborn as sns


Ensure that you are in the root folder of all the fold folders and target files
read_text(fold_name):
    fold_name: this is the name of the fold you want to read ALL patient files of. It will be read into a 2 dimensional
    list. If you would like to retrieve just the first patient instead, you will need to change the line 
    "txt_all.extend(txt[1:])" to "txt_all.append(txt[1:])" and you will be to use "read_text(fold1.txt)[0]" to retrieve
    the relevant patient's data
read_ans(file_name):
    file_name: this is the name of the file you want to read ALL targets of. It will be read into a 2 dimensional
    list. To retrieve the first patient's target: read_ans(ans.csv)[0]
put_single_into_dataframe(txt): This functions takes in 2 dimensional list ie the output of read_text(fold1.txt) 
put_multiple_into_dataframe(txt): Multiple is for using it with the output of read_text after you wanted to change it to append


In [2]:
def read_text(fold_name):
    txt_all = list()
    for f in os.listdir(fold_name): # for each file in the directory
        if f.endswith(".txt"):
            with open(os.path.join(fold_name, f), 'r') as fp: # open each file
                txt = fp.readlines() # read inside the file
                recordid = txt[1].rstrip('\n').split(',')[-1] # get recordid
                txt = [[int(recordid)] + t.rstrip('\n').split(',') for t in txt] # preface each row with the recordid as all patients are 1 file
                txt_all.extend(txt[1:]) # skip the parameter list
    return txt_all

def read_one_text(fold_name):
    txt_all = list()
    for f in os.listdir(fold_name): # for each file in the directory
        if f.endswith(".txt"):
            with open(os.path.join(fold_name, f), 'r') as fp: # open each file
                txt = fp.readlines() # read inside the file
            recordid = txt[1].rstrip('\n').split(',')[-1] # get recordid
            txt = [[int(recordid)] + t.rstrip('\n').split(',') for t in txt] # preface each row with the recordid as all patients are 1 file
            txt_all.append(txt[1:]) # skip the parameter list
    return txt_all

def read_ans(file_name):
    txt_all = list()
    with open(file_name, 'r') as fp: # opens the csv file
        txt = fp.readlines() 
    for i in range(1, len(txt)): # similar to above read_text
        record_id, length_of_stay, hospital_death = txt[i].rstrip('\n').split(',')
        txt_all.append([record_id, length_of_stay, hospital_death])
    return txt_all

def put_multiple_into_dataframe(txt_all):
    df = pd.DataFrame()
    for i in txt_all:
        df2 = pd.DataFrame(i, columns=['recordid', 'time', 'parameter', 'value'])
        df = df.append(df2, ignore_index=True)
    return df

def put_single_into_dataframe(txt_all):
    df = pd.DataFrame(txt_all, columns=['recordid', 'time', 'parameter', 'value'])
    return df

def get_X_add_ready(X_add, stat):
    X_add = X_add.reset_index()
    X_add = X_add.pivot(index='recordid', columns='parameter', values='value')
    X_add = X_add.drop(stat_feat, axis = 1) 
#     X_add = X_add.drop(['RecordID'], axis = 1) 
    X_add.columns = [x+stat for x in X_add.columns]
    X_add = X_add.reset_index()
    return X_add

<h1>1. Data Exploration</h1>
<p>Will in add explanation </p>

In [ ]:
p1 = read_one_text("../Project_Data/Fold1")[0]
p1df = pd.DataFrame(p1, columns=['recordid', 'time', 'parameter', 'value'])
df_1 = p1df.drop('recordid', axis=1)
df_1.drop(df_1.index[:1], inplace=True)
# df_1

df_1['value'] = pd.to_numeric(df_1['value'])
df_1.time = df_1.time.astype('str')
df_1['time'] = df_1['time'].str.replace(':', '.')
df_1['time'] = pd.to_numeric(df_1['time'])
df_1['time_value'] = list(zip(df_1.time, df_1.value))
df_2 = df_1.groupby('parameter').time_value.apply(lambda x: x.unique().tolist())

major_ticks = np.arange(0, 51, 10)
minor_ticks = np.arange(0, 49, 2)
fig = plt.figure(figsize=(18, 30))
ax = fig.add_subplot(1, 1, 1)
for i in range(len(df_2)):
    testList2 = [(elem1, elem2) for elem1, elem2 in df_2[i]]
    zip(*testList2)
    plt.plot(*zip(*testList2), marker='o', label=df_2.index[i])
plt.xlabel('Time (hours)')
plt.ylabel('Value')
plt.legend(loc='upper right')
# plt.xlim(xmin=0.0)
# plt.ylim(ymin=0.0)
ax.minorticks_on()
ax.set_xticks(minor_ticks, minor=True)

ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
# ax.tick_params(which = 'both', direction = 'out')
plt.show()

In [4]:
df_feat = pd.DataFrame()
numberOfFolds = 0
for i, name in enumerate(["Fold1"]): # what folds do you want to use?
    string = "../Project_Data/"
    string += name
    df_feat = df_feat.append(put_single_into_dataframe(read_text(string)))
    numberOfFolds = (i+1)
numberOfRows = numberOfFolds*1000
df_feat.head()

,recordid,time,parameter,value
0,132539,00:00,RecordID,132539
1,132539,00:00,Age,54
2,132539,00:00,Gender,0
3,132539,00:00,Height,-1
4,132539,00:00,ICUType,4


In [5]:
# Reading Target
df_target = pd.DataFrame(read_ans('../Project_Data/Fold1_Outcomes.csv'), columns=['recordid', 'days_in_hospital', 'mortality'])
df_target.head()

,recordid,days_in_hospital,mortality
0,132539,5,0
1,132540,8,0
2,132541,19,0
3,132543,9,0
4,132545,4,0


In [6]:

bin_feat = ['MechVent']
num_feat = ['Albumin', 'ALP', 'ALT', 'AST', 'Bilirubin', 'BUN', 'Cholesterol',
           'Creatinine', 'DiasABP', 'FiO2', 'GCS', 'Glucose', 'HCO3', 'HCT',
           'HR', 'K', 'Lactate', 'Mg', 'MAP', 'NA', 'NIDiasABP', 'NIMAP',
           'NISysABP', 'PaCO2', 'PaO2', 'pH', 'Platelets', 'RespRate', 'SaO2',
           'SysABP', 'Temp', 'Tropl', 'TropT', 'Urine', 'WBC', 'Weight']

print("Number of record ids:", len(df_feat['recordid'].unique()))
unique_count = df_feat['parameter'].value_counts()/numberOfRows
print(unique_count)

Number of record ids: 1000
HR             57.027
MAP            36.092
SysABP         35.979
DiasABP        35.955
Urine          34.208
Weight         33.679
NISysABP       24.457
NIDiasABP      24.424
NIMAP          24.088
Temp           21.204
GCS            15.214
RespRate       13.775
FiO2            7.815
MechVent        7.596
pH              5.770
PaO2            5.496
PaCO2           5.490
HCT             4.626
K               3.708
Creatinine      3.573
Platelets       3.566
BUN             3.547
HCO3            3.479
Mg              3.468
Na              3.462
Glucose         3.338
WBC             3.286
SaO2            1.985
Lactate         1.924
Height          1.000
ICUType         1.000
Age             1.000
RecordID        1.000
Gender          1.000
Bilirubin       0.858
AST             0.857
ALT             0.857
ALP             0.833
Albumin         0.617
TroponinT       0.566
TroponinI       0.130
Cholesterol     0.077
Name: parameter, dtype: float64


<h2>Analysis of Features</h2>
<p>The data above shows the average number of times a variable observed per patient. Based on the data above and the feature description we classify the features into these categories:
<ul>
    <li>General Descriptors (static data) that are collected when the patient is admitted to the ICU. Weight is not included as weight are measured multiple times as a time series data. Each of the descriptors will be included as a feature into the model.</li>
    <li>Rare features: measured on average less than one time per patient (less than 1.0). We use the <u>existence</u> of these measurements for each patient as a feature.</li>
    <li>Features that measured often or more that one time per patient (more than 1.0). Calculate the hourly average of each measurements and put them into 48 columns. <i>Example, average HR on the first hour to HR_1, average HR on the second hour to HR_2, and so on.</i></li>
</ul>
</p>


In [7]:
stat_feat = ['Age', 'Gender', 'Height', 'ICUType', 'RecordID'] #General Descriptors
rare_feat = []
nor_feat = []
for index, value in unique_count.items():
    if value < 1.0:
        rare_feat.append(index)
    elif index not in stat_feat:
        nor_feat.append(index)
rare_feat.append("MechVent")
print("Rare features", rare_feat)
print("Normal features", nor_feat)

Rare features ['Bilirubin', 'AST', 'ALT', 'ALP', 'Albumin', 'TroponinT', 'TroponinI', 'Cholesterol', 'MechVent']
Normal features ['HR', 'MAP', 'SysABP', 'DiasABP', 'Urine', 'Weight', 'NISysABP', 'NIDiasABP', 'NIMAP', 'Temp', 'GCS', 'RespRate', 'FiO2', 'MechVent', 'pH', 'PaO2', 'PaCO2', 'HCT', 'K', 'Creatinine', 'Platelets', 'BUN', 'HCO3', 'Mg', 'Na', 'Glucose', 'WBC', 'SaO2', 'Lactate']


In [8]:
# your code to produce test and train data
df = df_feat.copy()

<h2>Creation of Data Matrices</h2>
<p>We create 3 different matrices to convert temporal data into a matrix that is a single feature vector per patient </p>
<ul> 
    <li>First, in the cell below, we create a matrix that generalises a patient's attributes across the whole 48 hours, such as his max BUN measurement over the 48 hours. </li>
    <li>Second, Junhao what is your matrix? (Will add in soon)</li>
    <li></li>
</ul>

In [9]:
def preprocess_x_for_design_matrix_1(df_feat):
    non_bin_feat.clear()
    # your code to produce test and train data
    df = df_feat.copy()

    df['value'] = pd.to_numeric(df['value'])
    temp_df = df.loc[df['time'] == '00:00', :].copy() # get all the variables at time 0
    temp_df = temp_df.loc[temp_df['parameter'].isin(stat_feat)] # prune the dataframe to only those static variables
    temp_df = temp_df.pivot(index='recordid', columns='parameter', values='value') 
    temp_df = temp_df.reset_index()
    for i in temp_df: # for loop to change all the -1 values for static variables into np.nan
        idx = temp_df.index[temp_df[i] == -1].tolist()
        for j in idx:
            temp_df.loc[j, i] = np.nan
    final_df = temp_df.copy()

#     Dealing with rare_feat
    d = df_feat.groupby(['recordid', 'parameter'])[['value']].count()
    def specialFeature(special):
        id = []
        for index, row in d.iterrows():
            if index[1] == special:
                id.append(index[0])
        return id
    for x in rare_feat:
        id = specialFeature(x)
        final_df[x] = 0
        for i in id:
            for row in final_df.index:
                if row == i:
                    final_df.loc[row, x] = 1
    final_df = final_df.drop(["RecordID"],axis=1)

    # Getting the different attributes
    pd.set_option('display.max_columns', 500)
    temp_df = df.drop(df.index[df['parameter'].isin(rare_feat)].tolist())
    temp_df = temp_df.groupby(['recordid', 'parameter'])[['value']]
    for i in ['min', 'max', 'mean']: # the different parameters we will use
        if (i=='min'):
            X_add = temp_df.min() # get the min of each parameter
            X_add = get_X_add_ready(X_add, '_min')
            final_df = final_df.merge(X_add, left_on='recordid', right_on='recordid') # merge the min of the parameters to the final dataframe
        elif (i=='max'):
            X_add = temp_df.max() # get the max of each parameter
            X_add = get_X_add_ready(X_add, '_max')
            final_df = final_df.merge(X_add, left_on='recordid', right_on='recordid') # merge the min of the parameters to the final dataframe
        elif (i=='mean'):
            X_add = temp_df.mean() # get the mean of each parameter
            X_add = get_X_add_ready(X_add, '_mean')
            final_df = final_df.merge(X_add, left_on='recordid', right_on='recordid') # merge the min of the parameters to the final dataframe

    # dealing with ICUType categorical
    one_hot = pd.get_dummies(final_df['ICUType'])
    meaning_of_icu_types = {1:'Coronary Care Unit', 2: 'Cardiac Surgery Recovery Unit', 3: 'Medical ICU', 4: 'Surgical ICU'}
    one_hot.columns = [meaning_of_icu_types[x] for x in one_hot.columns]
    final_df = final_df.merge(one_hot, left_index=True, right_index=True)
    final_df = final_df.drop('ICUType', axis=1)
    
    # Extreme height values is set to np.nan
    for index, row in final_df.iterrows():
        if row["Height"] < 40 or row["Height"] > 210:
            row["Height"] = np.nan


    # Drop recordID column
    final_df = final_df.drop("recordid", axis=1)
            
    # Creating non binary column list and filling na values with mean
    for i in final_df:
        if i in rare_feat or i in bin_feat:
            continue
        final_df = final_df.fillna(final_df.mean())
        non_bin_feat.append(i)
        
#     display(final_df.head())
    return final_df

In [10]:
# Second Matrix
def preprocess_x_for_design_matrix_2(df):
    non_bin_feat.clear()
    
    df_static = df.loc[df['time'] == '00:00', :].copy()
    static_vars = ['RecordID', 'Age', 'Gender', 'Height', 'ICUType', 'Weight']
    df_static.drop('time', axis=1, inplace=True)
    df_static = df_static.loc[df_static['parameter'].isin(static_vars)]
    df_static = df_static.groupby(['recordid', 'parameter'])[['value']].last()
    df_static.reset_index(inplace=True)
    df_static = df_static.pivot(index='recordid', columns='parameter', values='value')

    for c in df_static.columns:
        df_static[c] = pd.to_numeric(df_static[c])

    for c in df_static.columns:
        x = df_static[c]
        if c == 'Height':
            idx = x < 0
            df_static.loc[idx, c] = np.nan
        elif c == 'Weight':
            idx = x < 0
            df_static.loc[idx, c] = np.nan
    d = df.groupby(['recordid', 'parameter'])[['value']].count()
    def specialFeature(special):
        id = []
        for index, row in d.iterrows():
            if index[1] == special:
                id.append(index[0])
        return id

    df2 = df_static.copy()
    df2.drop('RecordID', axis=1, inplace=True)

    for x in rare_feat:
        id = specialFeature(x)
        df2[x] = 0
        for i in id:
            for row in df2.index:
                if row == i:
                    df2.loc[row, x] = 1
    df2.head()
    normal_feat = nor_feat.copy()
    normal_feat.remove('MechVent')
    idx = df['parameter'].isin(normal_feat)
    df3 = df.loc[idx, :].copy()
    df3[['hour','min']] = df3.time.str.split(':', expand=True)
    df3["hour"] = pd.to_numeric(df3["hour"])
    df3["value"] = pd.to_numeric(df3["value"])
    bins = [0, 12, 24, 36, 48]
    labels = ['0', '12', '24', '36']
    df3 = df3.groupby(['recordid', pd.cut(df3.hour, bins=bins, labels=labels), 'parameter'])[['value']].mean()
    df3

    for n in normal_feat:    
        df2[n +'0'] = np.nan
        df2[n +'12'] = np.nan
        df2[n +'24'] = np.nan
        df2[n +'36'] = np.nan
    df2.head()

    for index, row in df3.iterrows():
        recordId = index[0]
        hour = index[1]
        parameter = index[2]
        df2.loc[recordId, parameter+hour] = row["value"]

    one_hot = pd.get_dummies(df2['ICUType'])
    meaning_of_icu_types = {1:'Coronary Care Unit', 2: 'Cardiac Surgery Recovery Unit', 3: 'Medical ICU', 4: 'Surgical ICU'}
    one_hot.columns = [meaning_of_icu_types[x] for x in one_hot.columns]
    df2 = df2.merge(one_hot, left_index=True, right_index=True)
    df2 = df2.drop('ICUType', axis=1)
    
    # Extreme height values is set to np.nan
    for index, row in df2.iterrows():
        if row["Height"] < 40 or row["Height"] > 210:
            row["Height"] = np.nan
            
    # Creating non binary column list and filling na values with mean
    for i in df2:
        if i in rare_feat or i in bin_feat:
            continue
        df2 = df2.fillna(df2.mean())
        non_bin_feat.append(i)

    display(df2.head())
    return df2

<h1>Model Building</h1>
<p>We develop models for our design matrix.</p>
<ul> 
    <li>First, in the cell below, we make use of the first design matrix above to pass in as input to the models. It mannually does cross validation on the 3 folds and validation on the remaining fold. Included is 3 regresssion models (Linear, DecisionTree, MLP) and 3 classification models (AdaboostedDecisionTrees, MLP, GaussianNB). Each utilises their own pipelines which makes use diffrent feature selectors </li>
    <li>Second, it utilises the pipelines introduced in First on the second design matrix</li>
    <li></li>
</ul>

In [ ]:
bin_feat = ["Gender", "Coronary Care Unit", "Cardiac Surgery Recovery Unit", "Medical ICU", "Surgical ICU"]
non_bin_feat = []

scaler = ColumnTransformer(
    remainder = 'passthrough',
    transformers=[
        ('num', StandardScaler(), non_bin_feat)])

parameters = {'VarianceThreshold':[0,0.5], 'PCA':[60,70],
              'SelectKBest':[80, 90],
              'DecisionTreeRegressor_min_samples_leaf':[1,3],
              'MLPRegressor_hiddenLayer':[(45,45,), (100,), (30,30,30)]
             }

all_list = [1,2,3,4]
for i in range(1,5):
    print("Testing on Fold", i)
    x_train_df = pd.DataFrame()
    y_train_df = pd.DataFrame()
    
    # Getting train data set up
    for j in [x for x in all_list if x != i]: 
        string = "../Project_Data/Fold"+str(j)
        y_file = "../Project_Data/Fold"+str(j)+"_Outcomes.csv"
        x_train_df = x_train_df.append(put_single_into_dataframe(read_text(string)))
        y_train_df = y_train_df.append(pd.DataFrame(read_ans(y_file), columns=['recordid', 'days_in_hospital', 'mortality']))
    x_train_df = preprocess_x_for_design_matrix_1(x_train_df)
    y_linear_train_df = y_train_df.drop(['recordid','mortality'], axis=1)
    for index, row in y_train_df.iterrows():
        if row['days_in_hospital'] == -1:
            row['days_in_hospital'] = 2
    y_linear_train_df['days_in_hospital'] = pd.to_numeric(y_linear_train_df['days_in_hospital'])
    y_classification_train_df = y_train_df.drop(['recordid','days_in_hospital'], axis=1)
    y_classification_train_df['mortality'] = pd.to_numeric(y_classification_train_df['mortality'])
        
    # Getting test data set up
    x_test_df = put_single_into_dataframe(read_text("../Project_Data/Fold"+str(i)))
    x_test_df = preprocess_x_for_design_matrix_1(x_test_df)
    y_test_df = pd.DataFrame(read_ans("../Project_Data/Fold"+str(i)+"_Outcomes.csv"), columns=['recordid', 'days_in_hospital', 'mortality'])
    y_linear_test_df = y_test_df.drop(['recordid','mortality'], axis=1)
    for index, row in y_test_df.iterrows():
        if row['days_in_hospital'] == -1:
            row['days_in_hospital'] = 2
    y_linear_test_df['days_in_hospital'] = pd.to_numeric(y_linear_test_df['days_in_hospital'])
    y_classification_test_df = y_test_df.drop(['recordid','days_in_hospital'], axis=1)
    y_classification_test_df['mortality'] = pd.to_numeric(y_classification_test_df['mortality'])
    
    # Create classification model and parameters to try out
    for nThreshold in parameters["VarianceThreshold"]:
        for nN_components in parameters["PCA"]:
            est = Pipeline(steps=[
                ('scaler', scaler),
                ('f_selecter', VarianceThreshold(threshold = nThreshold)),
                ('dim_reducer', PCA(n_components = nN_components)),
                ('classifier', AdaBoostClassifier())])
            est.fit(x_train_df, y_classification_train_df)
            prediction = est.predict(x_test_df)
            print ( "Using AdaBoostClassifier --> VarianceThreshold: {} and PCA: {} has a score of {}".format(nThreshold, nN_components, roc_auc_score(y_classification_test_df, prediction)) )
    for nThreshold in parameters["VarianceThreshold"]:
        for nN_components in parameters["PCA"]:
            for hiddenLayer in parameters["MLPRegressor_hiddenLayer"]:
                est = Pipeline(steps=[
                    ('scaler', scaler),
                    ('f_selecter', VarianceThreshold(threshold = nThreshold)),
                    ('dim_reducer', PCA(n_components = nN_components)),
                    ('classifier', MLPClassifier(hidden_layer_sizes = hiddenLayer))])
                est.fit(x_train_df, y_classification_train_df)
                prediction = est.predict(x_test_df)
                print ( "Using MLPClassifier --> HiddenLayer: {} VarianceThreshold: {} and PCA: {} has a score of {}".format(hiddenLayer, nThreshold, nN_components, roc_auc_score(y_classification_test_df, prediction)) )
    for nThreshold in parameters["VarianceThreshold"]:
        for nN_components in parameters["PCA"]:
            est = Pipeline(steps=[
                ('scaler', scaler),
                ('f_selecter', VarianceThreshold(threshold = nThreshold)),
                ('dim_reducer', PCA(n_components = nN_components)),
                ('classifier', GaussianNB())])
            est.fit(x_train_df, y_classification_train_df)
            prediction = est.predict(x_test_df)
            print ( "Using GaussianNB --> VarianceThreshold: {} and PCA: {} has a score of {}".format(nThreshold, nN_components, roc_auc_score(y_classification_test_df, prediction)) )        
    
    # Creating regression model and parameters to try out
    for nThreshold in parameters["VarianceThreshold"]:
        for nN_components in parameters["PCA"]:
            est = Pipeline(steps=[
            ('scaler', scaler),
            ('f_selecter', VarianceThreshold(threshold = nThreshold)),
            ('dim_reducer', PCA(n_components = nN_components)),
            ('classifier', LinearRegression())])
            est.fit(x_train_df, y_linear_train_df)
            prediction = est.predict(x_test_df)
            print ( "Using LinearRegression --> VarianceThreshold: {} and PCA: {} has a score of {}".format(nThreshold, nN_components, mean_squared_error(y_linear_test_df, prediction)) )
    for nThreshold in parameters["SelectKBest"]:
        for nN_components in parameters["PCA"]:
            for min_samples_leaf in parameters["DecisionTreeRegressor_min_samples_leaf"]:
                est = Pipeline(steps=[
                ('scaler', scaler),
                ('f_selecter', SelectKBest(k = 90)),
                ('dim_reducer', PCA(n_components = nN_components)),
                ('classifier', DecisionTreeRegressor(min_samples_leaf = min_samples_leaf))])
                est.fit(x_train_df, y_linear_train_df)
                prediction = est.predict(x_test_df)
                print ( "Using DecisionTreeRegressor({}) --> SelectKBest: 90 and PCA: {} has a score of {}".format(min_samples_leaf, nN_components, mean_squared_error(y_linear_test_df, prediction)) )
    for nThreshold in parameters["VarianceThreshold"]:
        for nN_components in parameters["PCA"]:
            for hiddenLayer in parameters["MLPRegressor_hiddenLayer"]:
                est = Pipeline(steps=[
                ('scaler', scaler),
                ('f_selecter', VarianceThreshold(threshold = nThreshold)),
                ('dim_reducer', PCA(n_components = nN_components)),
                ('classifier', MLPRegressor(hidden_layer_sizes = hiddenLayer, learning_rate_init = 0.01))])
                est.fit(x_train_df, y_linear_train_df)
                prediction = est.predict(x_test_df)
                print ( "Using MLPRegressor({}) --> SelectKBest: 90 and PCA: {} has a score of {}".format(hiddenLayer, nN_components, mean_squared_error(y_linear_test_df, prediction)) )
    print(end="\n\n")



In [11]:
bin_feat = ["Gender", "Coronary Care Unit", "Cardiac Surgery Recovery Unit", "Medical ICU", "Surgical ICU"]
non_bin_feat = []

scaler = ColumnTransformer(
    remainder = 'passthrough',
    transformers=[
        ('num', StandardScaler(), non_bin_feat)])

parameters = {'VarianceThreshold':[0,0.5], 'PCA':[60,70],
              'SelectKBest':[80, 90],
              'DecisionTreeRegressor_min_samples_leaf':[1,3],
              'MLPRegressor_hiddenLayer':[(45,45,), (100,), (30,30,30)]
             }

all_list = [1,2,3,4]
for i in range(1,5):
    print("Testing on Fold", i)
    x_train_df = pd.DataFrame()
    y_train_df = pd.DataFrame()
    
    # Getting train data set up
    for j in [x for x in all_list if x != i]: 
        string = "../Project_Data/Fold"+str(j)
        y_file = "../Project_Data/Fold"+str(j)+"_Outcomes.csv"
        x_train_df = x_train_df.append(put_single_into_dataframe(read_text(string)))
        y_train_df = y_train_df.append(pd.DataFrame(read_ans(y_file), columns=['recordid', 'days_in_hospital', 'mortality']))
    x_train_df = preprocess_x_for_design_matrix_2(x_train_df)
    y_linear_train_df = y_train_df.drop(['recordid','mortality'], axis=1)
    for index, row in y_train_df.iterrows():
        if row['days_in_hospital'] == -1:
            row['days_in_hospital'] = 2
    y_linear_train_df['days_in_hospital'] = pd.to_numeric(y_linear_train_df['days_in_hospital'])
    y_classification_train_df = y_train_df.drop(['recordid','days_in_hospital'], axis=1)
    y_classification_train_df['mortality'] = pd.to_numeric(y_classification_train_df['mortality'])
        
    # Getting test data set up
    x_test_df = put_single_into_dataframe(read_text("../Project_Data/Fold"+str(i)))
    x_test_df = preprocess_x_for_design_matrix_2(x_test_df)
    y_test_df = pd.DataFrame(read_ans("../Project_Data/Fold"+str(i)+"_Outcomes.csv"), columns=['recordid', 'days_in_hospital', 'mortality'])
    y_linear_test_df = y_test_df.drop(['recordid','mortality'], axis=1)
    for index, row in y_test_df.iterrows():
        if row['days_in_hospital'] == -1:
            row['days_in_hospital'] = 2
    y_linear_test_df['days_in_hospital'] = pd.to_numeric(y_linear_test_df['days_in_hospital'])
    y_classification_test_df = y_test_df.drop(['recordid','days_in_hospital'], axis=1)
    y_classification_test_df['mortality'] = pd.to_numeric(y_classification_test_df['mortality'])
        
    
    # Creating regression model and parameters to try out
    for nThreshold in parameters["VarianceThreshold"]:
        for nN_components in parameters["PCA"]:
            est = Pipeline(steps=[
            ('scaler', scaler),
            ('f_selecter', VarianceThreshold(threshold = nThreshold)),
            ('dim_reducer', PCA(n_components = nN_components)),
            ('classifier', LinearRegression())])
            est.fit(x_train_df, y_linear_train_df)
            prediction = est.predict(x_test_df)
            print ( "Using LinearRegression --> VarianceThreshold: {} and PCA: {} has a score of {}".format(nThreshold, nN_components, mean_squared_error(y_linear_test_df, prediction)) )
    for nThreshold in parameters["SelectKBest"]:
        for nN_components in parameters["PCA"]:
            for min_samples_leaf in parameters["DecisionTreeRegressor_min_samples_leaf"]:
                est = Pipeline(steps=[
                ('scaler', scaler),
                ('f_selecter', SelectKBest(k = 90)),
                ('dim_reducer', PCA(n_components = nN_components)),
                ('classifier', DecisionTreeRegressor(min_samples_leaf = min_samples_leaf))])
                est.fit(x_train_df, y_linear_train_df)
                prediction = est.predict(x_test_df)
                print ( "Using DecisionTreeRegressor({}) --> SelectKBest: 90 and PCA: {} has a score of {}".format(min_samples_leaf, nN_components, mean_squared_error(y_linear_test_df, prediction)) )
    for nThreshold in parameters["VarianceThreshold"]:
        for nN_components in parameters["PCA"]:
            for hiddenLayer in parameters["MLPRegressor_hiddenLayer"]:
                est = Pipeline(steps=[
                ('scaler', scaler),
                ('f_selecter', VarianceThreshold(threshold = nThreshold)),
                ('dim_reducer', PCA(n_components = nN_components)),
                ('classifier', MLPRegressor(hidden_layer_sizes = hiddenLayer, learning_rate_init = 0.01))])
                est.fit(x_train_df, y_linear_train_df)
                prediction = est.predict(x_test_df)
                print ( "Using MLPRegressor({}) --> SelectKBest: 90 and PCA: {} has a score of {}".format(hiddenLayer, nN_components, mean_squared_error(y_linear_test_df, prediction)) )
    print(end="\n\n")



Testing on Fold 1


,Age,Gender,Height,Weight,Bilirubin,AST,ALT,ALP,Albumin,TroponinT,...,SaO224,SaO236,Lactate0,Lactate12,Lactate24,Lactate36,Coronary Care Unit,Cardiac Surgery Recovery Unit,Medical ICU,Surgical ICU
recordid,,,,,,,,,,,,,,,,,,,,,
135076,56,1,177.800000,110.0,1,1,1,1,0,0,...,96.368618,96.356632,2.583674,2.30822,2.070263,2.008021,0,0,0,1
135077,72,1,169.685199,220.0,0,0,0,0,1,0,...,96.368618,96.356632,2.583674,2.30822,2.070263,2.008021,0,0,1,0
135079,68,1,169.685199,100.0,0,0,0,0,0,1,...,96.368618,96.356632,1.700000,2.30822,2.070263,2.008021,0,0,0,1
135080,77,1,170.200000,77.5,0,0,0,0,0,0,...,96.368618,96.356632,2.583674,2.80000,2.070263,2.008021,0,1,0,0
135081,66,1,162.600000,65.0,0,0,0,0,0,0,...,96.368618,96.356632,2.583674,2.30822,2.070263,2.008021,0,1,0,0


,Age,Gender,Height,Weight,Bilirubin,AST,ALT,ALP,Albumin,TroponinT,...,SaO224,SaO236,Lactate0,Lactate12,Lactate24,Lactate36,Coronary Care Unit,Cardiac Surgery Recovery Unit,Medical ICU,Surgical ICU
recordid,,,,,,,,,,,,,,,,,,,,,
132539,54,0,170.094476,81.422068,0,0,0,0,0,0,...,96.396396,96.604748,2.575443,2.395522,2.447271,2.711155,0,0,0,1
132540,76,1,175.300000,76.000000,0,0,0,0,0,0,...,96.396396,95.000000,2.575443,2.395522,2.447271,2.711155,0,1,0,0
132541,44,0,170.094476,56.700000,1,1,1,1,1,0,...,96.396396,96.604748,1.300000,1.900000,0.900000,2.711155,0,0,1,0
132543,68,1,180.300000,84.600000,1,1,1,1,1,0,...,96.396396,96.604748,2.575443,2.395522,2.447271,2.711155,0,0,1,0
132545,88,0,170.094476,81.422068,0,0,0,0,1,0,...,96.396396,96.604748,2.575443,2.395522,2.447271,2.711155,0,0,1,0


C:\Users\samue\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Using AdaBoostClassifier --> VarianceThreshold: 0 and PCA: 60 has a score of 0.6210511982570807


...

C:\Users\samue\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:1321: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Using MLPRegressor((100,)) --> SelectKBest: 90 and PCA: 70 has a score of 318.93088574069856


C:\Users\samue\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:1321: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Using MLPRegressor((30, 30, 30)) --> SelectKBest: 90 and PCA: 70 has a score of 253.7287779166953


Testing on Fold 2


KeyboardInterrupt: 